In [ ]:
# Data plotting
import matplotlib.pyplot as plt
# Data manipulation and file loading
import pandas
import numpy as np
import math
#Keras model and layers
from keras.models import Sequential, Model
from keras.layers import Dense,LSTM,Concatenate,Layer,Lambda,Input,Multiply,SimpleRNN,GRU,TimeDistributed
from sklearn.metrics import mean_squared_error
from keras import optimizers
import keras.backend
import tensorflow as tf
from keras import initializers      
# Preprocessing
from sklearn.preprocessing import MinMaxScaler

** Data Formatting **
Here, we convert the data to sequences with a "look-back" parameter. This simply creates sequences of such a size that are fed to the model to learn to predict the next value. The look-back is a parameter with the value being 7 days in this case.

In [ ]:
# Convert time-series data to LSTM training set

# The sequence over the entire dataset and one-step ahead value for prediction

# Perform a one-step ahead prediction over the entire sequence or look-back sized sequence 
def create_dataset(mode, ns, fname, look_back=1)
    dataX, dataY = [], []
    maxD = 0
    minD = 10000
    for c in ns:
        dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
        dataframe.dropna(how='all',inplace=True)
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        if np.max(dataset) > maxD:
            maxD = np.max(dataset)
        if np.min(dataset) < minD:
            minD = np.min(dataset)
        if (mode == 1):
            sql = len(dataset)
            dataX.append(dataset[:sql-1,0])
            dataY.append(dataset[1:sql,0])
        else:
            for i in range(len(dataset)-look_back):
                dataX.append(dataset[i:(i+look_back), 0])
                dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY), minD, maxD

# def create_dataset(ns, fname, look_back=1):
#     dataX, dataY = [], []
#     maxD = 0
#     minD = 10000
#     for c in ns:
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         dataset = dataset.astype('float32')
#         if np.max(dataset) > maxD:
#             maxD = np.max(dataset)
#         if np.min(dataset) < minD:
#             minD = np.min(dataset)
#         sql = len(dataset)
#         a = dataset[:sql-1,0]
#         b = dataset[1:sql,0]
#         dataX.append(a)
#         dataY.append(b)
#     return np.array(dataX), np.array(dataY), minD, maxD

# # One step ahead prediction for sequnces of smaller lengths of size lookback
# def create_dataset2(ns, fname, look_back=1):
#     for c in ns:
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         dataset = dataset.astype('float32')
#         if np.max(dataset) > maxD:
#             maxD = np.max(dataset)
#         if np.min(dataset) < minD:
#             minD = np.min(dataset)
#         for i in range(len(dataset)-look_back):
#             a = dataset[i:(i+look_back), 0]
#             dataX.append(a)
#             dataY.append(dataset[i + look_back, 0])
#     return np.array(dataX), np.array(dataY), minD, maxD

# Input data sequence normalized over the population and the duration

def create_dataset_2(mode, norm_flag, ns, fname, totalPop, ps, look_back=1):
    dataX, dataY = [], []
    for c in ns:
        pop = totalPop[int(c)-1]
        dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
        dataframe.dropna(how='all',inplace=True)
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        #dataset = dataset/pop
        #dataset = np.power(dataset,(1./ps))/40
        if (mode == 1):
            sql = len(dataset)
            a = dataset[:sql-1,0]
            b = dataset[1:sql,0]
            if norm_flag == 1:
                a = (np.sign(a)*np.abs(a)**(1./ps))/40
                b = (np.sign(b)*np.abs(b)**(1./ps))/40
            else:
                a = (np.sign(a)*np.abs(a)**(1./ps))
                b = (np.sign(b)*np.abs(b)**(1./ps))            
        else:
            for i in range(len(dataset)-look_back):
                a = dataset[i:(i+look_back), 0]
                b = dataset[i + look_back, 0]
                if norm_flag == 1:
                    a = (np.sign(a)*np.abs(a)**(1./ps))/40
                    b = (np.sign(b)*np.abs(b)**(1./ps))/40
                else:
                    a = (np.sign(a)*np.abs(a)**(1./ps))
                    b = (np.sign(b)*np.abs(b)**(1./ps))
            
        dataX.append(a)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)






# def create_dataset3(ns, fname, totalPop, ps, look_back=1):
#     dataX, dataY = [], []
#     for c in ns:
#         pop = totalPop[int(c)-1]
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         dataset = dataset.astype('float32')
#         #dataset = dataset/pop
#         #dataset = np.power(dataset,(1./ps))/40
#         sql = len(dataset)
#         a = dataset[:sql-1,0]
#         a = (np.sign(a)*np.abs(a)**(1./ps))/40
#         b = dataset[1:sql,0]
#         b = (np.sign(b)*np.abs(b)**(1./ps))/40
#         dataX.append(a)
#         dataY.append(b)
#     return np.array(dataX), np.array(dataY)

# # Input data sequence normalized over the population
# def create_dataset3p(ns, fname, totalPop, ps, look_back=1):
#     dataX, dataY = [], []
#     for c in ns:
#         pop = totalPop[int(c)-1]
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         dataset = dataset.astype('float32')
#         dataset = dataset/pop
#         #dataset = np.power(dataset,(1./ps))/40
#         sql = len(dataset)
#         a = dataset[:sql-1,0]
#         a = np.sign(a)*np.abs(a)**(1./ps)#/40
#         b = dataset[1:sql,0]
#         b = np.sign(b)*np.abs(b)**(1./ps)#/40
#         dataX.append(a)
#         dataY.append(b)
#     return np.array(dataX), np.array(dataY)

# # Input data sequence of lookback length normalized over the population and the duration
# def create_dataset4(ns, fname, totalPop, ps, look_back=1):
#     dataX, dataY = [], []
#     for c in ns:
#         pop = totalPop[int(c)-1]
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         #dataset = dataset.astype('float32')
#         #dataset = dataset/pop
#         #dataset = np.power(dataset,(1./ps))
#         for i in range(len(dataset)-look_back):
#             a = dataset[i:(i+look_back), 0]
#             #a = a/pop
#             b = dataset[i + look_back, 0]
#             a = (np.sign(a)*np.abs(a)**(1./ps))/40
#             b = (np.sign(b)*np.abs(b)**(1./ps))/40
#             dataX.append(a)
#             dataY.append(b)
#     return np.array(dataX), np.array(dataY)

# # Input data sequence of lookback length normalized over the population
# def create_dataset5(ns, fname, totalPop, ps, look_back=1):
#     dataX, dataY = [], []
#     for c in ns:
#         pop = totalPop[int(c)-1]
#         dataframe = pandas.read_csv(fname, usecols=[int(c)], engine='python')
#         dataframe.dropna(how='all',inplace=True)
#         dataset = dataframe.values
#         #dataset = dataset.astype('float32')
#         dataset = dataset/pop
#         #dataset = np.power(dataset,(1./ps))
#         for i in range(len(dataset)-look_back):
#             a = dataset[i:(i+look_back), 0]
#             #a = a/pop
#             b = dataset[i + look_back, 0]
#             a = np.sign(a)*np.abs(a)**(1./ps)#/40
#             b = np.sign(b)*np.abs(b)**(1./ps)#/40
#             dataX.append(a)
#             dataY.append(b)
#     return np.array(dataX), np.array(dataY)

In [ ]:
# Input columns from the csv file for the data
Loc = np.asarray([1,2,3,4,5,7,8,9,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28])

# Other set of input combinations that can be used 
#Loc = np.asarray([1,2,3,4,5,7,8,9,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,33,37,39,40,42,44,45,46,47,48,49,50,51])
#Loc = np.asarray([13,29,30,34,35,36,38,41,43])
#Loc = np.asarray(np.linspace(1,51,51)) 
#Loc = np.asarray([1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,36])

In [3]:
# Applying the formatting to the training set

dataframePop = pandas.read_csv('SinglePop.csv', engine='python')
datasetPop = dataframePop.values
datasetPop = datasetPop.astype('float32')
totalPop = datasetPop[0,:]

lb = 7
ps = 3
maxV = 40

trainActive, targetActive = create_dataset_2( 0, 1, Loc,'Active_cases_data.csv', totalPop, ps, lb)

trainTotal, targetTotal = create_dataset_2( 0, 1, Loc,'Total_cases_data.csv', totalPop, ps, lb)

train1d, nv = create_dataset_2( 0, 1, Loc,'One-day-change_data.csv', totalPop, ps, lb)

train2d, nv = create_dataset_2( 0, 1, Loc,'Three-day-change_data.csv', totalPop, ps, lb)

train3d, nv = create_dataset_2( 0, 1, Loc,'Seven-day-change_data.csv', totalPop, ps, lb)

# Minmax scaling for hospital density and population density
trainPop, nv, minPop, maxPop = create_dataset( 0, Loc,'Pop_data.csv', lb)
trainPop = (trainPop - minPop)/(maxPop - minPop)

trainMed, nv, minMed, maxMed = create_dataset( 0, Loc,'Med_data.csv',lb)
trainMed = (trainMed - minMed)/(maxMed - minMed)

trainPop2, nv, minPop2, maxPop2 = create_dataset( 0, Loc-1,'TotalPop.csv',lb)
trainPop2 = (trainPop2 - minPop2)/(maxPop2 - minPop2)


# Concatenate data into final train data format
numC = trainTotal.shape[0]
numT = trainTotal.shape[1]
trainX = np.concatenate((np.reshape(trainActive,(numC,numT,1)),np.reshape(trainTotal,(numC,numT,1)),np.reshape(train1d,(numC,numT,1)),np.reshape(train2d,(numC,numT,1)),np.reshape(train3d,(numC,numT,1)),np.reshape(trainPop,(numC,numT,1)),np.reshape(trainMed,(numC,numT,1)),np.reshape(trainPop2,(numC,numT,1))),axis=2)
#trainX = np.concatenate((np.reshape(trainTotal,(sql,lb,1)),np.reshape(trainPop,(sql,lb,1)),np.reshape(trainMed,(sql,lb,1))),axis=2)

trainY = np.concatenate((np.reshape(targetActive,(numC,1)),np.reshape(targetTotal,(numC,1))),axis=1)

NameError: name 'pandas' is not defined

** Model Definition**

Here, we define a custom LSTM architecture that integrates two temporal inputs and combines the two static attributes.

In [ ]:
# Creating the prediction arrays for different states
numR = 1# Number of runs
fcl = 180  # Number of days of prediction
totalPredict = np.zeros((1,fcl))
activePredict = np.zeros((1,fcl))

# Texas
totalPredictTx = np.zeros((1,fcl))
activePredictTx = np.zeros((1,fcl))

# Florida
totalPredictFl = np.zeros((1,fcl))
activePredictFl = np.zeros((1,fcl))

# New York
totalPredictNy = np.zeros((1,fcl))
activePredictNy = np.zeros((1,fcl))


for m_run in range(numR):
    keras.backend.clear_session()
    numT = 7   # number of timesteps  for look-back prediction
    numF = 2   # number of input channels (model1)
    numF2 = 8  # Batch size
    
    # Initialize the model 
    AllInput = Input(shape=(numT,numF2))
    RecInput = Lambda(lambda AllInput:AllInput[:,:,:numF],name='RecIn')(AllInput)
    HidInput = Lambda(lambda AllInput:AllInput[:,-1,5:7],name='HIn')(AllInput)
    H = LSTM(50, input_shape=(numT,numF),return_sequences=False)(RecInput)
    #DC = Dense(2,activation='relu')(H)
    D2 = (Dense(50,activation='sigmoid'))(HidInput)
    Data = Multiply()([H,D2])
    #D3 = (Dense(500,activation='sigmoid'))(Data)
    out = (Dense(2,activation='relu'))(Data)
    CoVid = Model(AllInput,out)
    #CoVid.summary()
    
    # The model is trained using mean squared error loss function and Adam Optimizer. Fit the model with the training data.
    ADopt = optimizers.SGD(lr=0.1)
    CoVid.compile(loss='mean_squared_error', optimizer=ADopt)
    CoVid.fit(trainX, trainY, epochs=100, batch_size=40, verbose=1)
 

    # Train the next model with a batch size of 8 .
    AllInputI = Input(batch_shape=(1,None,numF2))
    RecInputI = Lambda(lambda AllInputI:AllInputI[:,:,:numF],name='RecInI')(AllInputI)
    HidInputI = Lambda(lambda AllInputI:AllInputI[:,-1,5:7],name='HInI')(AllInputI)
    HI = LSTM(50, input_shape=(None,numF),return_sequences=False,stateful=False)(RecInputI)
    #DCI = Dense(2,activation='relu')(HI)
    D2I = Dense(50,activation='sigmoid')(HidInputI)
    DataI = Multiply()([HI,D2I])
    #D3I = (Dense(100,activation='sigmoid'))(DataI)
    outI = Dense(2,activation='relu')(DataI)

    CoVidI = Model(AllInputI,outI)
    #CoVidI.summary()

    # Transfer the pretrained weights to the new model.
    WT = CoVid.get_weights()
    CoVidI.set_weights(WT)
    
    # Set Bexar county data
    BexarPop = 1975000
    testPop =  1553

    # Scale population density and hospital bed density
    testMed =  (7839./BexarPop)*10000

    testPop = (testPop - minPop)/(maxPop - minPop)

    testMed = (testMed - minMed)/(maxMed - minMed)
    
    testPop2 = (BexarPop - minPop2)/(maxPop2-minPop2)

    # Set the test set for Bexar county
    testActive = np.asarray([29,29,39,45,56,68,81])
    testTotal = np.asarray([29,29,39,45,57,69,84])
    test1d = np.asarray([4,0,10,6,12,12,15])
    test2d = np.asarray([29,29,14,16,28,30,39])
    test3d = np.asarray([29,29,39,45,57,69,84])

    # Normalize and reshape the test data
    testActive = (np.sign(testActive) * np.abs(testActive)**(1./ps))/maxV
    testTotal = (np.sign(testTotal) * np.abs(testTotal)**(1./ps))/maxV
    test1d = (np.sign(test1d) * np.abs(test1d)**(1./ps))/maxV
    test2d = (np.sign(test2d) * np.abs(test2d)**(1./ps))/maxV
    test3d = (np.sign(test3d) * np.abs(test3d)**(1./ps))/maxV

    f1 = np.reshape(testActive,(1,lb,1))
    f2 = np.reshape(testTotal,(1,lb,1))
    f3 = np.reshape(test1d,(1,lb,1))
    f4 = np.reshape(test2d,(1,lb,1))
    f5 = np.reshape(test3d,(1,lb,1))
    f6 = np.reshape([testPop,testPop,testPop,testPop,testPop,testPop,testPop],(1,lb,1))
    f7 = np.reshape([testMed,testMed,testMed,testMed,testMed,testMed,testMed],(1,lb,1))
    f8 = np.reshape([testPop2,testPop2,testPop2,testPop2,testPop2,testPop2,testPop2],(1,lb,1))

    totalPredictT = np.zeros((1,fcl))
    activePredictT = np.zeros((1,fcl))

    totalGT = np.asarray([25,29,29,39,45,57,69,84])
    activeGT = np.asarray([25,29,29,39,45,56,68,81])

    initX = np.concatenate((f1,f2,f3,f4,f5,f6,f7,f8),axis=2)
    
    gtl = 7
    
    # Looping through duration of forecast for step ahead predictions
    for tp in range(7,fcl):
        if tp < gtl:
            singlePredict = CoVidI.predict(initX)
            # Store the predictions
            totalPredictT[0,tp] = np.power((np.float32(singlePredict[0][1]))*maxV,ps)
            activePredictT[0,tp] = np.power((np.float32(singlePredict[0][0]))*maxV,ps)
        else:
            prevX = initX
            oi = prevX[0,1:,:]
            singlePredict = CoVidI.predict(initX)

            prevT7 = np.power(prevX[0,0,1]*maxV,ps)

            prevT3 = np.power(prevX[0,4,1]*maxV,ps)

            prevT1 = np.power(prevX[0,-1,1]*maxV,ps)

            currentT = np.power(np.float32(singlePredict[0][1])*maxV,ps)

            f3 = np.max([0,(currentT - prevT1)])
            f4 = np.max([0,(currentT - prevT3)])
            f5 = np.max([0,(currentT - prevT7)])
            # Normalize
            f3 = (np.sign(f3) * np.abs(f3)**(1./ps))/maxV
            f4 = (np.sign(f4) * np.abs(f4)**(1./ps))/maxV
            f5 = (np.sign(f5) * np.abs(f5)**(1./ps))/maxV

            ci = np.reshape(prevX[0,-1,5:],(1,3))
            di = np.reshape(np.asarray([f3,f4,f5]),(1,3))
            completeIn = np.concatenate((singlePredict,di,ci),axis=1)
            initX = np.concatenate((oi,completeIn),axis=0)
            initX = np.reshape(initX,(1,lb,numF2))
            # Store the predictions
            totalPredictT[0,tp] = np.power((np.float32(singlePredict[0][1]))*maxV,ps)
            activePredictT[0,tp] = np.power((np.float32(singlePredict[0][0]))*maxV,ps)
            
    totalPredict = totalPredict + totalPredictT/numR
    activePredict = activePredict + activePredictT/numR